In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from itertools import islice
import arabicstopwords.arabicstopwords as stp

import pickle as pkl

import sentencepiece as sp

import tkseem as tk
import gensim

In [2]:
from arabert.preprocess import ArabertPreprocessor

model_name="bert-base-arabert"
arabert_prep = ArabertPreprocessor(model_name=model_name)

text = "ولن نبالغ إذا قلنا إن هاتف أو كمبيوتر المكتب في زمننا هذا ضروري"
arabert_prep.preprocess(text)

[2024-06-05 17:23:34,735 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


'و+ لن نبالغ إذا قل +نا إن هاتف أو كمبيوتر ال+ مكتب في زمن +نا هذا ضروري'

In [3]:
text = "هل غادر الشعراء من متردم أم هل عرفت الدار بعد توهم"
arabert_prep.preprocess(text)

'هل غادر ال+ شعراء من متردم أم هل عرف +ت ال+ دار بعد توهم'

In [4]:
text = "وتتابعت ذكرى الحبيب بخاطري والذكريات إذا جثمن جسام"
arabert_prep.preprocess(text)

'و+ تتابع +ت ذكرى ال+ حبيب ب+ خاطر +ي و+ ال+ ذكري +ات إذا جثمن جسام'

In [5]:
text = "أثني علي بما علمت فإنني سمح مخالطتي إذا لم أظلم"
arabert_prep.preprocess(text)

'أثني علي ب+ ما علم +ت ف+ إنني سمح مخالط +ت +ي إذا لم أظلم'

In [6]:
df = pd.read_csv('data/apcd_clean.csv')
df['poem_text'] = df['البيت'].astype(str)
df.drop('البيت', inplace=True, axis=1)
df.head()

,poem_text
0,خليلي لا تستعجلا أن تزودا وأن تجمعا شملي وت...
1,فما لبث يوما بسابق مغنم ولا سرعتي يوما بساب...
2,وإن تنظراني اليوم أقض لبانة وتستوجبا منا عل...
3,لعمرك ما نفس بجد رشيدة تؤامرني سرا لأصرم مرثدا
4,وإن ظهرت منه قوارص جمة وأفرع في لومي مرارا ...


In [7]:
def tokenize_text_bert(text):
    tokenized_text = arabert_prep.preprocess(text).split()
    return [token for token in tokenized_text if '+' not in token]

df['bert_t'] = df.poem_text.apply(tokenize_text_bert)
df.head()

,poem_text,bert_t
0,خليلي لا تستعجلا أن تزودا وأن تجمعا شملي وت...,"[خليل, لا, تستعجل, أن, تزود, أن, تجمع, شملي, ت..."
1,فما لبث يوما بسابق مغنم ولا سرعتي يوما بساب...,"[ما, لبث, يوم, سابق, مغنم, لا, سرع, يوم, سابق,..."
2,وإن تنظراني اليوم أقض لبانة وتستوجبا منا عل...,"[إن, تنظراني, يوم, أقض, بان, تستوجب, من, علي, ..."
3,لعمرك ما نفس بجد رشيدة تؤامرني سرا لأصرم مرثدا,"[عمر, ما, نفس, جد, رشيد, تؤامرني, سر, أصرم, مرثد]"
4,وإن ظهرت منه قوارص جمة وأفرع في لومي مرارا ...,"[إن, ظهر, من, قوارص, جم, أفرع, في, لومي, مرار,..."


In [20]:
bert_model = gensim.models.Word2Vec(
    df['bert_t'],
    window=8,
    min_count=3,
    epochs=10,
    workers=8)

In [21]:
bert_model.wv.most_similar(positive='حرب')

[('حروب', 0.8258386850357056),
 ('وغى', 0.7615727782249451),
 ('هيجاء', 0.7552218437194824),
 ('كفاح', 0.6719418168067932),
 ('هيجا', 0.6571804285049438),
 ('وغا', 0.6517561078071594),
 ('قتال', 0.6514930725097656),
 ('معارك', 0.6458837389945984),
 ('هياج', 0.6402440071105957),
 ('محارب', 0.6118369698524475)]

In [22]:
bert_model.wv.most_similar(positive='قمر')

[('بدر', 0.886574923992157),
 ('هلال', 0.7291861176490784),
 ('أقمار', 0.7184511423110962),
 ('قمرا', 0.6994655132293701),
 ('كوكب', 0.6927988529205322),
 ('بدرا', 0.6662076115608215),
 ('أبدر', 0.6478577256202698),
 ('إشراق', 0.6388406753540039),
 ('نير', 0.6296306848526001),
 ('بدران', 0.6207348108291626)]

In [23]:
bert_model.wv.most_similar(positive='سلاح')

[('اشحذ', 0.606364369392395),
 ('كمي', 0.6048628091812134),
 ('أسلح', 0.5965107679367065),
 ('يقاتل', 0.5614538192749023),
 ('مبارز', 0.5568662285804749),
 ('مدجج', 0.555096447467804),
 ('كمى', 0.5508106350898743),
 ('سلاحا', 0.541978120803833),
 ('أعزل', 0.5360102653503418),
 ('صائل', 0.5245010256767273)]

In [13]:
bert_model.wv.most_similar(positive='عيد')

[('فتتركني', 0.38507384061813354),
 ('ذاكا', 0.38497740030288696),
 ('ضئضىء', 0.3754619061946869),
 ('عقف', 0.3737567365169525),
 ('متحرف', 0.37313684821128845),
 ('شنفا', 0.3689161539077759),
 ('قلاق', 0.36809614300727844),
 ('مفحم', 0.36330750584602356),
 ('توعظ', 0.36208972334861755),
 ('سليس', 0.360828161239624)]

In [13]:
bert_model.wv.most_similar(positive='رمضان')

[('شعبان', 0.8914019465446472),
 ('شوال', 0.8817551732063293),
 ('رجب', 0.8156948089599609),
 ('صوم', 0.8102214336395264),
 ('صيام', 0.7968962788581848),
 ('إفطار', 0.7784942388534546),
 ('شهر', 0.7660092115402222),
 ('أسبوع', 0.711338460445404),
 ('سادس', 0.6981536149978638),
 ('صوما', 0.6931205987930298)]

In [14]:
bert_model.wv.most_similar(positive='صيام')

[('صوم', 0.9176177978515625),
 ('صوما', 0.8008038401603699),
 ('رمضان', 0.7968961596488953),
 ('شهر', 0.7709810733795166),
 ('إفطار', 0.7595511674880981),
 ('شعبان', 0.7455832362174988),
 ('صام', 0.7038789391517639),
 ('شوال', 0.7023992538452148),
 ('أفطر', 0.6908111572265625),
 ('افطر', 0.6848002076148987)]

In [15]:
bert_model.wv.most_similar(positive='نهر')

[('جدول', 0.8035427331924438),
 ('خليج', 0.75458163022995),
 ('دجل', 0.7471379637718201),
 ('أنهار', 0.7432496547698975),
 ('ضفاف', 0.7363280057907104),
 ('شاطئ', 0.7324143052101135),
 ('غدير', 0.7303998470306396),
 ('شاط', 0.7227619290351868),
 ('ماؤ', 0.7031472325325012),
 ('جداول', 0.6819625496864319)]

In [24]:
bert_model.wv.most_similar(positive='ثكلى')

[('ثاكل', 0.84678715467453),
 ('ثكول', 0.8041930198669434),
 ('ثواكل', 0.7970663905143738),
 ('تندب', 0.7553367018699646),
 ('تنوح', 0.7480713129043579),
 ('نوادب', 0.7214873433113098),
 ('نوائح', 0.7049776911735535),
 ('تبكي', 0.7032932639122009),
 ('انتحاب', 0.6990566849708557),
 ('مفجوع', 0.6974043846130371)]

In [25]:
bert_model.wv.most_similar(positive='حب')

[('حبك', 0.8353255391120911),
 ('هوا', 0.8068432807922363),
 ('هوى', 0.7780730724334717),
 ('محب', 0.7500185966491699),
 ('عشق', 0.744415283203125),
 ('حبي', 0.7434027791023254),
 ('غرام', 0.7380569577217102),
 ('هواي', 0.6968938708305359),
 ('شغف', 0.6701419353485107),
 ('عشقي', 0.6636521220207214)]

In [26]:
bert_model.wv.most_similar(positive='بكاء')

[('بكا', 0.9056922793388367),
 ('بكائي', 0.7947946190834045),
 ('بكاؤ', 0.7944477200508118),
 ('نحيب', 0.7501668334007263),
 ('انتحاب', 0.7455834746360779),
 ('بكائ', 0.7419630885124207),
 ('تأسف', 0.7137796878814697),
 ('تفجع', 0.6928139328956604),
 ('بكي', 0.692628026008606),
 ('يبكى', 0.6859516501426697)]

In [27]:
bert_model.wv.most_similar(positive='شرق')

[('شرقا', 0.7292301058769226),
 ('غرب', 0.7085141539573669),
 ('مشارق', 0.7072569727897644),
 ('مشرق', 0.6671646237373352),
 ('غربا', 0.6281417608261108),
 ('مغارب', 0.6271656155586243),
 ('مشرقا', 0.5730795860290527),
 ('المغرب', 0.5716384649276733),
 ('مغرب', 0.5604066252708435),
 ('مغاربا', 0.5396143198013306)]

In [28]:
bert_model.wv.most_similar(positive='تهجد')

[('متهجد', 0.7273576855659485),
 ('يتهجد', 0.7060710787773132),
 ('أذكار', 0.6124588251113892),
 ('آناء', 0.6017740368843079),
 ('ليلاء', 0.5946472883224487),
 ('نهارا', 0.5937535762786865),
 ('تلاو', 0.5759906768798828),
 ('سهر', 0.5758674144744873),
 ('صوما', 0.5733753442764282),
 ('متنفل', 0.5731920003890991)]

In [29]:
bert_model.wv.most_similar(positive='عجوز')

[('شمطاء', 0.783287763595581),
 ('عانس', 0.6760879755020142),
 ('شهرب', 0.6484708189964294),
 ('حبلى', 0.6186768412590027),
 ('صبي', 0.6026730537414551),
 ('الصبي', 0.5987450480461121),
 ('مومس', 0.5935264825820923),
 ('حليل', 0.5896438360214233),
 ('است', 0.5808508992195129),
 ('تزوج', 0.5726931691169739)]

In [22]:
bert_model.wv.most_similar(positive='شيطان')

[('رجيم', 0.8654313683509827),
 ('نزغ', 0.7513377666473389),
 ('غوي', 0.7489640712738037),
 ('إبليس', 0.7442300915718079),
 ('مارد', 0.6984922289848328),
 ('ابليس', 0.6911853551864624),
 ('غاوي', 0.6905670166015625),
 ('شيطاني', 0.6768575310707092),
 ('شياطين', 0.6654265522956848),
 ('غاو', 0.6376013159751892)]

In [30]:
bert_model.wv.most_similar(positive='سفيه')

[('سفي', 0.7586504817008972),
 ('مأفون', 0.708583652973175),
 ('سفه', 0.6947180032730103),
 ('أحمق', 0.6945162415504456),
 ('سفهاء', 0.693919837474823),
 ('جهول', 0.6899445056915283),
 ('حليم', 0.6749348044395447),
 ('سفاه', 0.6733116507530212),
 ('حمق', 0.6719596982002258),
 ('سخيف', 0.6596214771270752)]

In [24]:
bert_model.wv.most_similar(positive='ليت')

[('فيالي', 0.7545331716537476),
 ('ياليت', 0.6638481616973877),
 ('يتني', 0.6485655307769775),
 ('نسيه', 0.5457009673118591),
 ('حسرتا', 0.5044820308685303),
 ('ويح', 0.504289984703064),
 ('يلتا', 0.4894065260887146),
 ('أتغفل', 0.4734174907207489),
 ('أترى', 0.46551939845085144),
 ('أسفي', 0.45672500133514404)]

In [31]:
bert_model.wv.most_similar(positive='رائد')

[('مرتاد', 0.6523571610450745),
 ('رواد', 0.618211567401886),
 ('منتجع', 0.5527411699295044),
 ('نجع', 0.5491498112678528),
 ('مرعى', 0.5381015539169312),
 ('ارتياد', 0.5214452743530273),
 ('يرتاد', 0.4937489926815033),
 ('يرود', 0.48257604241371155),
 ('مورد', 0.4771170914173126),
 ('انتجاع', 0.4717232882976532)]

In [32]:
bert_model.wv.most_similar(positive='شجاع')

[('جبان', 0.7316989898681641),
 ('باسل', 0.7143270373344421),
 ('ذمر', 0.7069920897483826),
 ('جريء', 0.7066757082939148),
 ('بسال', 0.6984866261482239),
 ('كمي', 0.6909453272819519),
 ('بطل', 0.682129442691803),
 ('إقدام', 0.6651972532272339),
 ('صنديد', 0.6500799655914307),
 ('مقدام', 0.6471388339996338)]

In [27]:
bert_model.wv.most_similar(positive='عنتر')

[('عبسي', 0.8010635375976562),
 ('أشجع', 0.7444644570350647),
 ('فارس', 0.7316497564315796),
 ('مغوار', 0.7279079556465149),
 ('جساس', 0.7232443690299988),
 ('فوارس', 0.7115820646286011),
 ('كبش', 0.7102588415145874),
 ('وغا', 0.7037143707275391),
 ('محرب', 0.6948537230491638),
 ('ضيغما', 0.6913682818412781)]

In [33]:
bert_model.save('models/bert_word2vec_model.model')
df.to_csv('all_poems_bert_processed.csv', index=False)

In [29]:
df.head()

,poem_text,bert_t
0,خليلي لا تستعجلا أن تزودا وأن تجمعا شملي وت...,"[خليل, لا, تستعجل, أن, تزود, أن, تجمع, شملي, ت..."
1,فما لبث يوما بسابق مغنم ولا سرعتي يوما بساب...,"[ما, لبث, يوم, سابق, مغنم, لا, سرع, يوم, سابق,..."
2,وإن تنظراني اليوم أقض لبانة وتستوجبا منا عل...,"[إن, تنظراني, يوم, أقض, بان, تستوجب, من, علي, ..."
3,لعمرك ما نفس بجد رشيدة تؤامرني سرا لأصرم مرثدا,"[عمر, ما, نفس, جد, رشيد, تؤامرني, سر, أصرم, مرثد]"
4,وإن ظهرت منه قوارص جمة وأفرع في لومي مرارا ...,"[إن, ظهر, من, قوارص, جم, أفرع, في, لومي, مرار,..."


## Doc2Vec

In [8]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
taggedData = [TaggedDocument(words=doc, tags=[str(i)]) for doc, i in zip(df['bert_t'], df.index)]

In [9]:
model = Doc2Vec(vector_size=150, min_count=4, epochs=20, workers=120, window=7, dm=1)
model

In [10]:
model.build_vocab(taggedData)

In [ ]:
model.train(taggedData,
            total_examples=model.corpus_count,
            epochs=model.epochs)

In [ ]:
#df['doc_vecs'] = df['bert_t'].apply(model.infer_vector)
df.head()

In [ ]:
#df.to_csv('all_poems_bert_processed.csv', index=False)

In [ ]:
model.save('models/bert_doc2vec_model_bow11.model')

In [ ]:
prompt = 'وددت رؤية الياسمين'

clean_prompt = [token for token in arabert_prep.preprocess(prompt).split() if '+' not in token]
clean_prompt

In [ ]:
inferred_vector = model.infer_vector(clean_prompt)
sims = model.dv.most_similar([inferred_vector], topn=500)
sims

In [ ]:
array = np.array(sims)
df_ggl = pd.DataFrame(array, columns=['id', 'similarity'])
plt.plot(df_ggl.index, df_ggl['similarity'].astype(float))

In [ ]:
df.loc[int(sims[2][0])]